# Transformer with mlp decoder

* Reference: https://pytorch.org/tutorials/beginner/transformer_tutorial.html

In [1]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
from torch.utils.data import dataset

def generate_square_subsequent_mask(sz: int) -> Tensor:
    """
    Generates an upper-triangular matrix of -int, with zeros on dialog
    refer to https://pytorch.org/docs/stable/generated/torch.triu.html
    """
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float=0.1, max_len: int=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
        
    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        # here we also add a dropout layer for position encoding
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int, nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Tansformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        # should be replaced with TransformerDeCoder
        self.decoder = nn.Linear(d_model, ntoken)
        self.init_weights()
        
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
        
    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len
            
        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [2]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk'])
vocab.set_default_index(vocab['<unk'])

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """
    converts raw text into a flat tensor
    """
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

# train_iter was "consumed" by the process of building the vocab, so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batchify(data: Tensor, bsz: int) -> Tensor:
    """
    Divides the data into bsz separate sequences, removing extra elements that wouldn't clearly fit.
    
    Args:
        data: Tensor, shape [N]
        bsz: int, batch size
        
    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    # rounding up based on batch size
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    # transform to bsz * seq_len, then use contiguous() to return a contiguous in memory tensor 
    # containing the same data as self tensor. If self tensor is already in the specified memory format, 
    # this function returns the self tensor: refer to https://pytorch.org/docs/stable/generated/torch.Tensor.contiguous.html?highlight=contiguous#torch.Tensor.contiguous
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size) # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

In [3]:
bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        sources: Tensor, shape [full_seq_len, batch_size]
        i: int
        
    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [4]:
ntokens = len(vocab)  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

In [5]:
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        batch_size = data.size(0)
        if batch_size != bptt:  # only on last batch
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += batch_size * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [6]:
%%time
best_val_loss = float('inf')
epochs = 20
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 14.41 | loss  8.12 | ppl  3345.60
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 11.41 | loss  6.88 | ppl   975.14
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 11.39 | loss  6.44 | ppl   625.56
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 11.42 | loss  6.31 | ppl   549.52
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 11.38 | loss  6.18 | ppl   482.50
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 11.15 | loss  6.16 | ppl   471.07
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 11.29 | loss  6.12 | ppl   454.07
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 11.26 | loss  6.11 | ppl   450.68
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 11.32 | loss  6.02 | ppl   413.18
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 11.27 | loss  6.02 | ppl   410.86
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 11.29 | loss  5.90 | ppl   364.56
| epoch   1 |  2400/ 

| epoch   6 |  1800/ 2928 batches | lr 3.87 | ms/batch 11.84 | loss  5.14 | ppl   170.26
| epoch   6 |  2000/ 2928 batches | lr 3.87 | ms/batch 11.81 | loss  5.13 | ppl   169.44
| epoch   6 |  2200/ 2928 batches | lr 3.87 | ms/batch 11.83 | loss  5.01 | ppl   149.19
| epoch   6 |  2400/ 2928 batches | lr 3.87 | ms/batch 11.82 | loss  5.12 | ppl   166.66
| epoch   6 |  2600/ 2928 batches | lr 3.87 | ms/batch 11.82 | loss  5.13 | ppl   169.61
| epoch   6 |  2800/ 2928 batches | lr 3.87 | ms/batch 11.83 | loss  5.07 | ppl   158.42
-----------------------------------------------------------------------------------------
| end of epoch   6 | time: 36.16s | valid loss  5.58 | valid ppl   266.15
-----------------------------------------------------------------------------------------
| epoch   7 |   200/ 2928 batches | lr 3.68 | ms/batch 11.88 | loss  5.11 | ppl   165.71
| epoch   7 |   400/ 2928 batches | lr 3.68 | ms/batch 11.83 | loss  5.13 | ppl   168.80
| epoch   7 |   600/ 2928 batches 

| epoch  12 |   200/ 2928 batches | lr 2.84 | ms/batch 11.92 | loss  4.76 | ppl   117.13
| epoch  12 |   400/ 2928 batches | lr 2.84 | ms/batch 11.98 | loss  4.78 | ppl   119.10
| epoch  12 |   600/ 2928 batches | lr 2.84 | ms/batch 12.17 | loss  4.61 | ppl   100.91
| epoch  12 |   800/ 2928 batches | lr 2.84 | ms/batch 12.19 | loss  4.68 | ppl   107.67
| epoch  12 |  1000/ 2928 batches | lr 2.84 | ms/batch 12.17 | loss  4.67 | ppl   106.19
| epoch  12 |  1200/ 2928 batches | lr 2.84 | ms/batch 12.18 | loss  4.71 | ppl   111.31
| epoch  12 |  1400/ 2928 batches | lr 2.84 | ms/batch 12.18 | loss  4.72 | ppl   111.93
| epoch  12 |  1600/ 2928 batches | lr 2.84 | ms/batch 12.18 | loss  4.76 | ppl   116.62
| epoch  12 |  1800/ 2928 batches | lr 2.84 | ms/batch 12.18 | loss  4.73 | ppl   113.53
| epoch  12 |  2000/ 2928 batches | lr 2.84 | ms/batch 12.18 | loss  4.72 | ppl   112.21
| epoch  12 |  2200/ 2928 batches | lr 2.84 | ms/batch 12.22 | loss  4.58 | ppl    97.45
| epoch  12 |  2400/ 

| epoch  17 |  1800/ 2928 batches | lr 2.20 | ms/batch 11.86 | loss  4.52 | ppl    91.63
| epoch  17 |  2000/ 2928 batches | lr 2.20 | ms/batch 11.86 | loss  4.50 | ppl    90.39
| epoch  17 |  2200/ 2928 batches | lr 2.20 | ms/batch 11.85 | loss  4.36 | ppl    78.39
| epoch  17 |  2400/ 2928 batches | lr 2.20 | ms/batch 11.86 | loss  4.48 | ppl    88.05
| epoch  17 |  2600/ 2928 batches | lr 2.20 | ms/batch 11.86 | loss  4.49 | ppl    89.52
| epoch  17 |  2800/ 2928 batches | lr 2.20 | ms/batch 11.85 | loss  4.44 | ppl    85.15
-----------------------------------------------------------------------------------------
| end of epoch  17 | time: 36.27s | valid loss  5.55 | valid ppl   258.45
-----------------------------------------------------------------------------------------
| epoch  18 |   200/ 2928 batches | lr 2.09 | ms/batch 11.92 | loss  4.50 | ppl    89.99
| epoch  18 |   400/ 2928 batches | lr 2.09 | ms/batch 11.86 | loss  4.51 | ppl    91.19
| epoch  18 |   600/ 2928 batches 